# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'01-06-2022'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'01-06-2022'

In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2022-01-07 04:22:09,33.93911,67.709953,158275,7367,NaN,NaN,Afghanistan,406.580388,4.654557
1,NaN,NaN,NaN,Albania,2022-01-07 04:22:09,41.15330,20.168300,213257,3228,NaN,NaN,Albania,7410.417680,1.513667
2,NaN,NaN,NaN,Algeria,2022-01-07 04:22:09,28.03390,1.659600,220825,6318,NaN,NaN,Algeria,503.579812,2.861089
3,NaN,NaN,NaN,Andorra,2022-01-07 04:22:09,42.50630,1.521800,25289,141,NaN,NaN,Andorra,32730.214198,0.557555
4,NaN,NaN,NaN,Angola,2022-01-07 04:22:09,-11.20270,17.873900,87625,1799,NaN,NaN,Angola,266.610739,2.053067


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,12/28/21,12/29/21,12/30/21,12/31/21,1/1/22,1/2/22,1/3/22,1/4/22,1/5/22,1/6/22
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,157998,158037,158056,158084,158107,158189,158183,158205,158245,158275
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,208352,208899,208899,210224,210224,210885,210885,212021,212021,213257
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,217265,217647,218037,218432,218818,219159,219532,219953,220415,220825


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,12/28/21,12/29/21,12/30/21,12/31/21,1/1/22,1/2/22,1/3/22,1/4/22,1/5/22,1/6/22
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,7355,7356,7356,7356,7356,7361,7364,7365,7367,7367
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,3207,3212,3212,3217,3217,3220,3220,3224,3224,3228
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,6254,6263,6271,6276,6284,6291,6298,6303,6310,6318


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,12/28/21,12/29/21,12/30/21,12/31/21,1/1/22,1/2/22,1/3/22,1/4/22,1/5/22,1/6/22
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,12/28/21,12/29/21,12/30/21,12/31/21,1/1/22,1/2/22,1/3/22,1/4/22,1/5/22,1/6/22
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,10828,10886,10953,11018,11018,11184,11256,11347,11478,11638
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,39084,39298,39583,39911,39911,40396,40549,40838,41312,41855
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,3750,3777,3819,3860,3860,3932,3961,3999,4036,4101


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,12/28/21,12/29/21,12/30/21,12/31/21,1/1/22,1/2/22,1/3/22,1/4/22,1/5/22,1/6/22
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,160,160,160,160,160,160,160,161,161,161
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,593,593,593,593,593,593,593,594,594,598
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,81,81,81,81,81,81,81,81,81,83


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
673,1001.0,Autauga,Alabama,US,2022-01-07 04:22:09,32.539527,-86.644082,11638,161,NaN,NaN,"Autauga, Alabama, US",20830.872219,1.383399
710,1075.0,Lamar,Alabama,US,2022-01-07 04:22:09,33.779950,-88.096680,2650,60,NaN,NaN,"Lamar, Alabama, US",19195.943499,2.264151
711,1077.0,Lauderdale,Alabama,US,2022-01-07 04:22:09,34.901719,-87.656247,16613,320,NaN,NaN,"Lauderdale, Alabama, US",17915.646669,1.926202


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,58487665,833987,0.0
India,35109286,482876,0.0
Brazil,22328252,619654,0.0


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,58487665,833987,0.0,57653678.0,2022-01-07 04:22:09,37.934269,-91.444083
India,35109286,482876,0.0,34626410.0,2022-01-07 04:22:09,23.088275,81.806127
Brazil,22328252,619654,0.0,21708598.0,2022-01-07 04:22:09,-12.669522,-48.480493
United Kingdom,14100750,149993,0.0,13950757.0,2022-01-07 04:22:09,28.164647,-33.966498
France,11290010,126002,0.0,11164008.0,2022-01-07 04:22:09,6.430808,-34.730285


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,5894729,76791,0.0
Texas,4897362,76260,0.0
Florida,4486276,62622,0.0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,5894729,76791,0.0,5817938.0,2022-01-07 04:22:09,37.843962,-120.728594
Texas,4897362,76260,0.0,4821102.0,2022-01-07 04:22:09,31.660643,-98.653069
Florida,4486276,62622,0.0,4423654.0,2022-01-07 04:22:09,28.940755,-82.700744
New York,3890666,60166,0.0,3830500.0,2022-01-07 04:22:09,42.544151,-75.474183
Illinois,2339534,30693,0.0,2308841.0,2022-01-07 04:22:09,39.843618,-89.178330


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,1843922,27728,0.0
Arizona,Maricopa,904822,13849,0.0
Illinois,Cook,900993,12159,0.0
Florida,Miami-Dade,854670,6472,0.0
Texas,Harris,718772,9792,0.0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,1843922,27728,0.0,1816194.0,2022-01-07 04:22:09,34.308284,-118.228241,6037.0
Arizona,Maricopa,904822,13849,0.0,890973.0,2022-01-07 04:22:09,33.348359,-112.491815,4013.0
Illinois,Cook,900993,12159,0.0,888834.0,2022-01-07 04:22:09,41.841448,-87.816588,17031.0
Florida,Miami-Dade,854670,6472,0.0,848198.0,2022-01-07 04:22:09,25.611236,-80.551706,12086.0
Texas,Harris,718772,9792,0.0,708980.0,2022-01-07 04:22:09,29.858649,-95.393395,48201.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2022-01-04,158205,212021,219953,24802,84666,4486,5820536,345126,608803,1294325,...,13723275,420108,199271,7,445369,1800704,471090,10143,265479,217678
2022-01-05,158245,212021,220415,25289,86636,4486,5915695,345255,684229,1304086,...,13918510,425436,199361,7,445680,1817721,471414,10146,269991,219057
2022-01-06,158275,213257,220825,25289,87625,4715,6025303,345389,762458,1312939,...,14100750,430958,199507,7,445680,1843563,471414,10152,274087,220178


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2022-01-04,7365,3224,6303,141,1778,119,117294,7986,2290,13784,...,149417,6180,1485,1,5337,33245,4947,1984,3762,5078
2022-01-05,7367,3224,6310,141,1789,119,117346,7989,2302,13802,...,149761,6181,1485,1,5341,33475,4954,1985,3772,5092
2022-01-06,7367,3228,6318,141,1799,119,117386,7993,2321,13823,...,149993,6186,1485,1,5341,33644,4954,1986,3782,5108


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2022-01-04,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2022-01-05,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2022-01-06,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,12/28/21,12/29/21,12/30/21,12/31/21,1/1/22,1/2/22,1/3/22,1/4/22,1/5/22,1/6/22
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,10828,10886,10953,11018,11018,11184,11256,11347,11478,11638
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,2586,2595,2603,2613,2613,2617,2624,2625,2634,2650
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,15537,15637,15768,15863,15863,16034,16084,16154,16396,16613
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,4976,5009,5046,5071,5071,5130,5145,5174,5231,5295
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,26454,26595,26789,26975,26975,27323,27438,27892,28208,28620


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-01-04,921175,160089,10,1411813,581134,5768815,962777,544468,191649,49,...,1462996,4804721,656407,68957,10714,1176159,888336,339679,1142290,117389
2022-01-05,932250,161678,10,1419562,588622,5808171,971525,554812,192968,49,...,1462996,4851987,663654,69762,11160,1186887,899036,342607,1156050,118098
2022-01-06,944876,161678,10,1430241,596409,5894729,979437,563635,195600,49,...,1462996,4897362,672567,71942,11833,1202727,913980,347554,1170504,119093


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                      \
Admin2         Autauga Baldwin Barbour  Bibb Blount Bullock Butler Calhoun   
2022-01-04       11347   40838    3999  4616  11430    1724   3695   23801   
2022-01-05       11478   41312    4036  4680  11497    1752   3773   24047   
2022-01-06       11638   41855    4101  4730  11587    1783   3838   24282   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2022-01-04         6402     3448  ...    5711   1589     6074     1537   
2022-01-05         6467     3487  ...    5731   1590     6111     1542   
2022-01-06         6603     3511  ...    5747   1595     6139     1547   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2022-01-04           8388  6452  4191          0     1892   1257  
2022-01-05           8423  6581  4220          0     1903   1260  
2022-01-06           8471  6836  4245          0     1909   1261  

[3 rows x 3336 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-01-04,16496,979,0,24509,9248,76634,10335,9274,2286,0,...,20941,76031,3811,480,89,15622,9909,5361,11246,1572
2022-01-05,16503,979,0,24570,9256,76760,10354,9281,2286,0,...,20941,76132,3855,482,89,15631,9909,5372,11319,1572
2022-01-06,16543,979,0,24586,9278,76791,10402,9281,2286,0,...,20941,76260,3868,482,89,15642,9974,5392,11376,1572


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2022-01-04         161     594      81   95    198      46    102     532   
2022-01-05         161     594      81   95    198      46    102     532   
2022-01-06         161     598      83   95    198      46    102     532   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2022-01-04          147       66  ...     121     37       63       24   
2022-01-05          147       66  ...     121     37       63       24   
2022-01-06          147       66  ...     121     37       63       24   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2022-01-04            110    14    34          0       39     16  
2022-01-05            110    14    34          0       39     16  
2022-01-06            110    14    34          0       39     16  

[3 rows x 3336 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2022-01-04,0.000139,0.005387,0.001918,0.012244,0.010768,0.047397,0.014150,0.000261,0.133061,0.004264,...,0.016726,0.007526,0.000447,0.0,0.000645,0.012214,0.002857,0.000493,0.016300,0.007363
2022-01-05,0.000253,0.000000,0.002100,0.019636,0.023268,0.000000,0.016349,0.000374,0.123892,0.007541,...,0.014227,0.012682,0.000452,0.0,0.000698,0.009450,0.000688,0.000296,0.016996,0.006335
2022-01-06,0.000190,0.005830,0.001860,0.000000,0.011416,0.051048,0.018528,0.000388,0.114332,0.006789,...,0.013093,0.012980,0.000732,0.0,0.000000,0.014217,0.000000,0.000591,0.015171,0.005117


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2022-01-04,0.000136,0.001242,0.000794,0.007143,0.001690,0.0,0.000418,0.000376,0.008811,0.001817,...,0.000335,0.000486,0.0,0.0,0.000375,0.006784,0.005692,0.000000,0.002398,0.006142
2022-01-05,0.000272,0.000000,0.001111,0.000000,0.006187,0.0,0.000443,0.000376,0.005240,0.001306,...,0.002302,0.000162,0.0,0.0,0.000749,0.006918,0.001415,0.000504,0.002658,0.002757
2022-01-06,0.000000,0.001241,0.001268,0.000000,0.005590,0.0,0.000341,0.000501,0.008254,0.001522,...,0.001549,0.000809,0.0,0.0,0.000000,0.005049,0.000000,0.000504,0.002651,0.003142


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2022-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-01-04,0.008288,0.000000,0.0,0.005135,0.011421,0.013490,0.008673,0.019859,0.011971,0.0,...,0.006349,0.011269,0.007152,0.025749,0.041205,0.015012,0.007019,0.006975,0.008773,0.006396
2022-01-05,0.012023,0.009926,0.0,0.005489,0.012885,0.006822,0.009086,0.018998,0.006882,0.0,...,0.000000,0.009837,0.011040,0.011674,0.041628,0.009121,0.012045,0.008620,0.012046,0.006040
2022-01-06,0.013544,0.000000,0.0,0.007523,0.013229,0.014903,0.008144,0.015903,0.013640,0.0,...,0.000000,0.009352,0.013430,0.031249,0.060305,0.013346,0.016622,0.014439,0.012503,0.008425


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2022-01-04      0.008085  0.007127  0.009594  0.004789  0.005454  0.009959   
2022-01-05      0.011545  0.011607  0.009252  0.013865  0.005862  0.016241   
2022-01-06      0.013940  0.013144  0.016105  0.010684  0.007828  0.017694   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2022-01-04      0.006264  0.004516  0.012494  0.005541  ...  0.002457   
2022-01-05      0.021110  0.010336  0.010153  0.011311  ...  0.003502   
2022-01-06      0.017228  0.009773  0.021030  0.006883  ...  0.002792   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2022-01-04      0.002524  0.004465  0.004575   0.004310  0.028535  0.004313   
2022-01-05      0.000629  0.006092  0.003253   0.004173  0.019994  0.006920   
2022-01-06      0.003145  0.004582  0.003243   0.005699  0.038748  0.005924   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2022-01-04            NaN  0.003181  0.000000  
2022-01-05            NaN  0.005814  0.002387  
2022-01-06            NaN  0.003153  0.000794  

[3 rows x 3336 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-01-04,0.002492,0.0,NaN,0.006323,0.002928,0.001228,0.004080,0.006949,0.0,NaN,...,0.002297,0.000842,0.001840,0.000000,0.0,0.000448,0.005684,0.000934,0.005454,0.030144
2022-01-05,0.000424,0.0,NaN,0.002489,0.000865,0.001644,0.001838,0.000755,0.0,NaN,...,0.000000,0.001328,0.011546,0.004167,0.0,0.000576,0.000000,0.002052,0.006491,0.000000
2022-01-06,0.002424,0.0,NaN,0.000651,0.002377,0.000404,0.004636,0.000000,0.0,NaN,...,0.000000,0.001681,0.003372,0.000000,0.0,0.000704,0.006560,0.003723,0.005036,0.000000


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State  Alabama                                                 \
Admin2          Autauga   Baldwin   Barbour Bibb Blount Bullock Butler   
2022-01-04      0.00625  0.001686  0.000000  0.0    0.0     0.0    0.0   
2022-01-05      0.00000  0.000000  0.000000  0.0    0.0     0.0    0.0   
2022-01-06      0.00000  0.006734  0.024691  0.0    0.0     0.0    0.0   

Province_State                             ...   Wyoming                      \
Admin2         Calhoun Chambers  Cherokee  ...      Park    Platte  Sheridan   
2022-01-04         0.0      0.0  0.015385  ...  0.034188  0.057143  0.032787   
2022-01-05         0.0      0.0  0.000000  ...  0.000000  0.000000  0.000000   
2022-01-06         0.0      0.0  0.000000  ...  0.000000  0.000000  0.000000   

Province_State                                                            \
Admin2          Sublette Sweetwater Teton     Uinta Unassigned  Washakie   
2022-01-04      0.043478   0.028037   0.0  0.096774        NaN  0.054054   
2022-01-05      0.000000   0.000000   0.0  0.000000        NaN  0.000000   
2022-01-06      0.000000   0.000000   0.0  0.000000        NaN  0.000000   

Province_State            
Admin2            Weston  
2022-01-04      0.142857  
2022-01-05      0.000000  
2022-01-06      0.000000  

[3 rows x 3336 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2022-01-04,0.000145,0.003320,0.001799,0.015379,0.010759,0.023800,0.010199,0.000188,0.109466,0.003435,...,0.014854,0.005448,0.000442,3.973643e-08,0.000490,0.010734,0.001444,0.000493,0.012639,0.006748
2022-01-05,0.000199,0.001660,0.001950,0.017507,0.017014,0.011900,0.013274,0.000281,0.116679,0.005488,...,0.014540,0.009065,0.000447,1.986821e-08,0.000594,0.010092,0.001066,0.000395,0.014817,0.006542
2022-01-06,0.000194,0.003745,0.001905,0.008754,0.014215,0.031474,0.015901,0.000334,0.115505,0.006139,...,0.013817,0.011022,0.000590,9.934107e-09,0.000297,0.012154,0.000533,0.000493,0.014994,0.005829


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2022-01-04,0.000257,0.000817,0.000960,0.003601,0.001517,0.000133,0.000341,0.000457,0.005900,0.001243,...,0.000449,0.000416,0.000019,0.0,0.000406,0.006579,0.002871,0.000001,0.002152,0.004989
2022-01-05,0.000264,0.000408,0.001035,0.001801,0.003852,0.000067,0.000392,0.000416,0.005570,0.001274,...,0.001376,0.000289,0.000009,0.0,0.000578,0.006749,0.002143,0.000253,0.002405,0.003873
2022-01-06,0.000132,0.000825,0.001152,0.000900,0.004721,0.000033,0.000367,0.000459,0.006912,0.001398,...,0.001462,0.000549,0.000005,0.0,0.000289,0.005899,0.001071,0.000378,0.002528,0.003508


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2022-01-04,-0.5,-0.499907,-0.496523,-0.498569,-0.493288,-0.499171,-0.498809,-0.49966,-0.499402,-0.499731,...,-0.481849,-0.499758,-0.496783,-0.5,-0.498237,-0.453466,-0.499858,-0.497938,-0.498727,-0.489078
2022-01-05,-0.5,-0.499907,-0.496523,-0.498569,-0.493288,-0.499171,-0.498809,-0.49966,-0.499402,-0.499731,...,-0.481849,-0.499758,-0.496783,-0.5,-0.498237,-0.453466,-0.499858,-0.497938,-0.498727,-0.489078
2022-01-06,-0.5,-0.499907,-0.496523,-0.498569,-0.493288,-0.499171,-0.498809,-0.49966,-0.499402,-0.499731,...,-0.481849,-0.499758,-0.496783,-0.5,-0.498237,-0.453466,-0.499858,-0.497938,-0.498727,-0.489078


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-01-04,0.007570,0.004706,1.091394e-11,0.005924,0.007846,0.014215,0.011738,0.021998,0.013386,1.423164e-199,...,0.010763,0.014771,0.009521,0.024187,0.031170,0.017348,0.013484,0.010679,0.007439,0.005463
2022-01-05,0.009796,0.007316,5.456968e-12,0.005707,0.010366,0.010519,0.010412,0.020498,0.010134,7.115821e-200,...,0.005382,0.012304,0.010281,0.017930,0.036399,0.013235,0.012764,0.009649,0.009743,0.005751
2022-01-06,0.011670,0.003658,2.728484e-12,0.006615,0.011797,0.012711,0.009278,0.018201,0.011887,3.557910e-200,...,0.002691,0.010828,0.011856,0.024590,0.048352,0.013290,0.014693,0.012044,0.011123,0.007088


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2022-01-04      0.007873  0.006464  0.009548  0.004904  0.004601  0.008834   
2022-01-05      0.009709  0.009036  0.009400  0.009385  0.005231  0.012538   
2022-01-06      0.011824  0.011090  0.012753  0.010034  0.006530  0.015116   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2022-01-04      0.006191  0.004469  0.010751  0.005235  ...  0.002586   
2022-01-05      0.013650  0.007402  0.010452  0.008273  ...  0.003044   
2022-01-06      0.015439  0.008587  0.015741  0.007578  ...  0.002918   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2022-01-04      0.002228  0.003675  0.002808   0.003172  0.028320  0.003330   
2022-01-05      0.001429  0.004883  0.003031   0.003672  0.024157  0.005125   
2022-01-06      0.002287  0.004733  0.003137   0.004685  0.031453  0.005525   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2022-01-04           -1.0  0.002433  0.000650  
2022-01-05           -1.0  0.004123  0.001518  
2022-01-06           -1.0  0.003638  0.001156  

[3 rows x 3336 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-01-04,0.001260,0.000274,NaN,0.003637,0.002696,0.000743,0.002794,0.004870,0.000055,NaN,...,0.001891,0.001195,0.002095,0.004967,0.000178,0.000812,0.002890,0.001545,0.003152,0.015073
2022-01-05,0.000842,0.000137,NaN,0.003063,0.001780,0.001194,0.002316,0.002812,0.000028,NaN,...,0.000946,0.001262,0.006820,0.004567,0.000089,0.000694,0.001445,0.001798,0.004821,0.007536
2022-01-06,0.001633,0.000068,NaN,0.001857,0.002079,0.000799,0.003476,0.001406,0.000014,NaN,...,0.000473,0.001472,0.005096,0.002283,0.000044,0.000699,0.004002,0.002761,0.004929,0.003768


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State   Alabama                                                 \
Admin2           Autauga   Baldwin       Barbour          Bibb   Blount   
2022-01-04      0.003125  0.000843  1.862647e-10  1.239722e-12  0.00004   
2022-01-05      0.001563  0.000422  9.313235e-11  6.198610e-13  0.00002   
2022-01-06      0.000781  0.003578  1.234568e-02  3.099305e-13  0.00001   

Province_State                                                                \
Admin2               Bullock        Butler   Calhoun      Chambers  Cherokee   
2022-01-04      3.311370e-10  2.953635e-10  0.000016  8.952691e-07  0.007694   
2022-01-05      1.655685e-10  1.476818e-10  0.000008  4.476346e-07  0.003847   
2022-01-06      8.278424e-11  7.384089e-11  0.000004  2.238173e-07  0.001924   

Province_State  ...   Wyoming                                           \
Admin2          ...      Park    Platte  Sheridan  Sublette Sweetwater   
2022-01-04      ...  0.017096  0.028573  0.016394  0.021739   0.014019   
2022-01-05      ...  0.008548  0.014287  0.008197  0.010870   0.007009   
2022-01-06      ...  0.004274  0.007143  0.004098  0.005435   0.003505   

Province_State                                                         
Admin2                 Teton     Uinta Unassigned  Washakie    Weston  
2022-01-04      2.545172e-25  0.048387   0.752941  0.027027  0.071429  
2022-01-05      1.272586e-25  0.024194   0.752941  0.013514  0.035714  
2022-01-06      6.362931e-26  0.012097   0.752941  0.006757  0.017857  

[3 rows x 3336 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210307,AK,56886,NaN,NaN,NaN,totalTestsViral,1731628,33.0,1293.0,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
1,20210307,AL,499819,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788,494.0,45976.0,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2,20210307,AR,324818,69092.0,2480716.0,NaN,totalTestsViral,2736442,335.0,14926.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
3,20210307,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
4,20210307,AZ,826454,56519.0,3073010.0,NaN,totalTestsViral,7908105,963.0,57907.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210307,AK,56886.0,NaN,NaN,NaN,totalTestsViral,1731628.0,33.0,1293.0,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
1,20210307,AL,499819.0,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788.0,494.0,45976.0,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2,20210307,AR,324818.0,69092.0,2480716.0,NaN,totalTestsViral,2736442.0,335.0,14926.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
3,20210307,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
4,20210307,AZ,826454.0,56519.0,3073010.0,NaN,totalTestsViral,7908105.0,963.0,57907.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-03-07,AK,56886,NaN,NaN,NaN,totalTestsViral,1731628,33.0,1293.0,NaN,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
2021-03-07,AL,499819,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788,494.0,45976.0,NaN,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2021-03-07,AR,324818,69092.0,2480716.0,NaN,totalTestsViral,2736442,335.0,14926.0,141.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
2021-03-07,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
2021-03-07,AZ,826454,56519.0,3073010.0,NaN,totalTestsViral,7908105,963.0,57907.0,273.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-03-07,AK,56886.0,NaN,NaN,NaN,totalTestsViral,1731628.0,33.0,1293.0,NaN,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
2021-03-07,AL,499819.0,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788.0,494.0,45976.0,NaN,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2021-03-07,AR,324818.0,69092.0,2480716.0,NaN,totalTestsViral,2736442.0,335.0,14926.0,141.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
2021-03-07,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
2021-03-07,AZ,826454.0,56519.0,3073010.0,NaN,totalTestsViral,7908105.0,963.0,57907.0,273.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


#### 